# Просепт

Можно считать эту тетрадку главной(хотя и много чернового кода), главная функция есть (допиливаем предсказание классификатором)

Вопросы которые интересовали задали в ридми файле, но так же по ходу буду сопровождать вопросами короткими и длинными комментариями

**То что было определено в начале и изменялось по ходу работы**

Важные моменты:
1) +Убрать название заказчика из названий от дилеров
2) +Проверить названия на всех платформах, и брать все уникальные слова, что есть на платформах (по каждому названию в таблице продукции)
3) +Соединять данные по последней таблице разметки, с нее же делить на выборки
4) ? 2 варианта: обучать по таблице с разметкой (как? Собрать свою метрику Андрей предлаал) или без учителя (сравнивать с ними же, но диллеров)
5) Выдавать не чисто 5 вариантов, а те варианты, которые проходят какой то порог, либо сделать настраиваемый. (В ТЗ указано, что это должно быть настраиваемым, значит первыми выводить наиболее вероятные)
6) -Пайплайн или нет? Возможно
7) -Подходит ли для задачи BERT? Андрей пробует
8) -Умеет ли кто работать с классами? 
9) -Градиентный бустинг в какой момент? 

Работа с текстом:

0) +очистка от знаков, один регистр
1) +Лемматизация
2) Мешок слов
3) N-граммы
4) +TF-IDF



Ход работы:

0) Формат вывода для бэкэнда (допустим передаем лучшие 10 в порядке убывания, а пользователь через интерфейс сам выбирает сколько выводить)
1) +Очистка, преобразование текста, Собрать уникальные слова в названиях (от заказчика)
2) ?собрать датасет для МО
3) +Утвердить или разработать метрику оценки
4) проверить разные стратегии: обучить или по наименьшему расстояни между векторами
5) Сравнить стратегии/модели
6) Завернуть в функцию/ функции

Ира:
1) +Изменение чисел на строки, очистка от прочих знаков
2) +Убрать просепт в названиях
3) +Лемматизация текста
4) пайплайн на будущее

Денис:
1) +Регулярное выражение на изменение единиц измерения объема, выделить их в отдельную фичу
2) +Собрать уникальные слова с названий всех платформ
3) +Выдача первого результата если есть артикул

Андрей:
1) пробует BERT
2) TF-IDF

In [1]:
!pip install pandas
!pip install numpy
!pip install spacy
!pip install faiss-cpu
!pip install nltk
!pip install scikit-learn

In [ ]:
!python -m spacy download ru_core_news_sm
!python -m spacy download en_core_web_sm

In [2]:
import numpy as np
import matplotlib
import pandas as pd
import re
from matplotlib import pyplot as plt
import spacy
from collections import OrderedDict

from scipy.spatial.distance import pdist, squareform


import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from nltk.corpus import stopwords as nltk_stopwords
import faiss

In [4]:
def csv_reading(path='', prep=True):
    # Функция загрузки данных. по умолчани берет файли из директории и проводит препроцессинг
    # Если установить prep=False то загрузит сырые данные как они есть
    dealers = pd.read_csv(path+'marketing_dealer.csv', sep=';')
    parser = pd.read_csv(path+'marketing_dealerprice.csv', sep=';')
    products = pd.read_csv(path+'marketing_product.csv', sep=';')
    matches = pd.read_csv(path+'marketing_productdealerkey.csv', sep=';')
    if prep==True:
        dealers = dealers.sort_values('id').reset_index(drop=True) #Сортируем диллеров по id 
        parser = parser.drop(['id','product_url', 'date'], axis=1).drop_duplicates().rename(columns={'product_key': 'key'}).reset_index(drop=True)
        products = products.drop(['Unnamed: 0', 'ean_13','category_id', 'ozon_article', 'wb_article', 'ym_article', 'wb_article_td'], axis=1)
        matches = matches.drop(['id','dealer_id'], axis=1)
    return dealers, parser, products, matches

In [5]:
dealers, parser, products, matches = csv_reading()

**Функции**

За основной признак мы будем брать наименования товаров, которые предлагают производитель и диллеры. Из этих признаков мы будем извлекать полезную информацию. Например: объем продукции, единицы измерения, а также артикулы (если они есть). Также преобразуем текст названий товаров

In [6]:
def text_worker(name): #отдельно функция для предварительной очистки текста
    #добавляем пробел на явных стыках и переходах
    pattern = [r"([а-я])([a-zA-ZА-Я])", r"([А-Я])([A-Za-z])", r"([a-z])([A-Zа-яА-Я])"]     
    for p in pattern:
        try:
            name = re.sub(p, "\\1 \\2", name)
        except:
            pass
    name = name.lower()
    bad = ['просепт','prosept50','prosept50,', 'prosepteco50','prosepteco50,','ultra', 'crystal', '-ая', 'prosept','ф/п']
    good = ['prosept',' prosept50 ',' prosept50 ', ' prosepteco50 ', ' prosepteco50 ', ' ultra ', ' crystal ', '', '', 'пакет']
    for o,n in zip(bad, good):
        name = name.replace(o,n)
    return name

In [7]:
def change_equal(name): 
    '''Функция принимает текст и пытается вытащить из него 
    артитул, еденицы измерения, количество (переводя в одну единицу измерения), текст очищеный от предыдущих элементов
    и все это возвращает в види словаря'''
    name = text_worker(name)
    dictionary={}
    try:
        article = re.search(r' \d+-\d+/?\d?[а-я]?',name)[0].strip()
        name = name.replace(article,'')
    except:
        article = np.nan
    try:
        v = re.findall(r'\d+(?:[\.,]\d+)? ?(?:мл|кг|г|л|шт)',name)[-1]
        dimension = re.search(r'[млкгшт]+',v)[0]
        quantity = float(v.replace(dimension,'').replace(',','.'))
        if dimension == 'мл':
            quantity = quantity/1000
            dimension = 'л'
        elif dimension == 'г':
            quantity = quantity/1000
            dimension = 'кг'
        name = name.replace(v,'')
        #for _v_ in v
        #    name = name.replace(_v_,'')
    except:
        dimension = np.nan
        quantity = 0
    finally:
        dictionary['name_new'] = ' '.join(re.sub(r'\W+', ' ', name).split(r'\W+'))
        dictionary['article'] = article
        dictionary['dimension'] = dimension
        dictionary['quantity'] = quantity
        return dictionary        

Лемматизировать текст необходимо для более правильной векторизации текста

In [8]:
def lemmatizate(text, nlp, nlp_ru):
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc])
    doc_ru = nlp_ru(text)
    return " ".join([token.lemma_ for token in doc_ru])

In [9]:
def parser_prep(parser):
    # работаем с таблицей работы парсера
    parser_new = parser['product_name'].apply(lambda x: change_equal(x)).apply(pd.Series)
    parser_final = pd.concat([parser, parser_new],axis=1).drop(['product_name'], axis=1)
    parser_final['article'] = parser_final['article'].fillna('')
    
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    nlp_ru = spacy.load('ru_core_news_sm', disable=['parser', 'ner'])
    parser_final['name_new'] = parser_final['name_new'].apply(lambda x: lemmatizate(x,nlp, nlp_ru))
    
    return parser_final

Также мы предположили, что название товара у производителя можно собрать из разных колонок `name_1c`, `ozon_name`, `name`, `wb_name`, чтобы извлечь как можно больше информации из товаров производителя

In [10]:
def products_prep(products): # выдает финальную версию таблицы продукции
    #заполняем пропуски
    name_columns = ['name_1c', 'ozon_name', 'name', 'wb_name']
    products['name_1c'] = products['name_1c'].fillna(products['name'])
    products['ozon_name'] = products['ozon_name'].fillna(products['name_1c'])
    products[name_columns] = products[name_columns].fillna('')
    
    #Из субъективно лучшей колонки названия вытаскиваем количество и ед.измерения из остальных только очищеное имя
    name_1c = products['name_1c'].apply(change_equal).apply(pd.Series).rename(columns={'name_new': 'name_1c_new'})['name_1c_new']
    ozon_name = products['ozon_name'].apply(change_equal).apply(pd.Series).drop(['article'],axis=1).rename(columns={'name_new': 'ozon_name_new'})
    name_1 = products['name'].apply(change_equal).apply(pd.Series).rename(columns={'name_new': 'name_new'})['name_new']
    wb_name = products['wb_name'].apply(change_equal).apply(pd.Series).rename(columns={'name_new': 'wb_name_new'})['wb_name_new']
    products1 = pd.concat([products, ozon_name, name_1c, wb_name, name_1],axis=1)
    
    #Соединяем названия из 4х колонок в full name (только уникальные слова в том же порядке)
    products1['full_name'] = products1.apply(lambda x: ' '.join(list(dict.fromkeys((x['name_1c_new']+' '+x['ozon_name_new']+' '+x['name_new']+' '+x['wb_name_new']).split()))), axis=1)
    
    #лемматизируем
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    nlp_ru = spacy.load('ru_core_news_sm', disable=['parser', 'ner'])
    products1['full_name'] = products1['full_name'].apply(lambda x: lemmatizate(x,nlp, nlp_ru))
    
    products1['article'] = products1['article'].apply(lambda x: x.lower())
    
    #Собираем финальную версию таблицы продукции
    products_final = products1.loc[:,['id','article','cost','recommended_price', 'full_name', 'dimension', 'quantity']]
    
    #заполняем пропуски рекомендованной цены медианой
    products_final['recommended_price'] = products_final['recommended_price'].fillna(products_final['recommended_price'].median())
    return products_final

In [36]:
def find_product_id_by_article(article):
    id = []
    if type(article) == '':
        return np.nan
    else:
        #print(f'Артикул {article}, будет поиск')
        try:
            id.append(int(list(products_final.loc[products_final['article']==article].id)[0])) #fr"'.*{article}.*'"
        except:
            return np.nan
        return id

In [12]:
def accuracy_aggregator(targets, idx, base_index,n):
    acc = 0
    for target, el in zip(targets.values.tolist(), idx.tolist()):
        acc += int(target in [base_index[r] for r in el[:n]])
    result = 100 * acc / len(idx)
    return result

In [14]:
def fit_tf_idf(products_final,parser_final):
    korpus = pd.concat([products_final['full_name'], parser_final['name_new']])
    nltk.download('stopwords')
    stopwords=set(nltk_stopwords.words('russian'))
    count_tf_idf = TfidfVectorizer(stop_words=list(stopwords))
    count_tf_idf.fit(korpus)
    return count_tf_idf

In [15]:
def search_neighbors(base_tfidf, query_tfidf):
    dims = base_tfidf.shape[1]
    n_cells = 1 # количество центроидов
    quantizer = faiss.IndexFlatL2(dims)
    idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
    # подготовка к поиску
    idx_l2.train(np.ascontiguousarray(base_tfidf.values).astype('float32'))
    idx_l2.add(np.ascontiguousarray(base_tfidf.values).astype('float32'))
    # создание словаря для нахождения индекса товара в базовом наборе данных
    base_index = {k: v for k, v in enumerate(base_tfidf.index.to_list())}
    vecs, idx = idx_l2.search(np.ascontiguousarray(query_tfidf.values).astype('float32'), 10)
    return base_index, vecs, idx

In [23]:
def get_neighbors(products_final, parser_final, matches=''):
    #Если передаем таблицу мэтчей - значит будет показана метрика
    #Создадим две таблицы. Первая: base - данные производителя, где индексы - это id товаров. Вторая: train - соответствие запроса к парсеру и правильного ответа к продукции
    base = products_final.copy().set_index('id').drop(['cost'],axis=1)
    parser_matches = parser_final.merge(matches, how='right',left_on='key',right_on='key').dropna(subset=['name_new']).drop_duplicates().groupby('key').first().reset_index()
    parser_matches = parser_matches[['key','price', 'name_new', 'article','quantity','dimension','product_id']]
    targets = parser_matches["product_id"]
    query = parser_matches.drop("product_id", axis=1)
    
    count_tf_idf = fit_tf_idf(products_final, parser_final)
    
    base_tfidf = pd.DataFrame(count_tf_idf.transform(base['full_name']).toarray()).set_index(base.index)
    query_tfidf = pd.DataFrame(count_tf_idf.transform(query['name_new']).toarray())
    base_tfidf.columns = base_tfidf.columns.astype('str')
    query_tfidf.columns = query_tfidf.columns.astype('str')
    
    base_index, vecs, idx = search_neighbors(base_tfidf, query_tfidf)
    
    dict_of_neighbors={}
    for k,v in zip(query['key'], idx.tolist()):
        list_of_preds = [base_index[i] for i in v]
        dict_of_neighbors[k] = list_of_preds
    
    return dict_of_neighbors, targets

In [80]:
#dict_of_neigh.update(d)
def dict_join(dict_all, dict_small):
    for key in dict_small.keys():
        #print('Ключ',key)
        #print('Список для этого ключа в исходном словаре',dict_of_neigh[key])
        l1 = dict_all[key]
        l2 = dict_small[key]
        l_all = list(dict.fromkeys(l1 + l2))
        #print(f'В общем словаре по ключу {key} значения: {l1}, в новом такие:{l2}, вместе: {l_all}')
        #print('Замена в этом списке:',dict_of_neigh[key])
        dict_all[key] = l_all
        #print('После замены:',dict_of_neigh[key])
    return dict_all

In [81]:
#функция рабочая, но не доделали ()
def main_function(path=''):
    # загрузка документов
    dilers , parser, products, matches = csv_reading(path)
    # преобразование текста(обе таблицы, лемматизация)
    parser_final = parser_prep(parser)
    products_final = products_prep(products)
    #Составляем макет словаря
    dict_of_neigh = dict.fromkeys(parser_final['key'],[])
    
    #Поиск по артикулу
    dict_of_article_neigh = (pd.DataFrame(parser_final['article']
                                           .apply(find_product_id_by_article))
                              .set_index(parser_final['key'])
                              .dropna().to_dict()['article'])
    dict_of_neigh.update(product_id_from_article_dict)
    
    # Tf-idf
    dict_of_tfidf_neighbors, targets = get_neighbors(products_final, parser_final, matches)
    
    # сборка каждый к каждому(query-base)
    
    # классификация через модель
    # отбор топ-n по predict-proba
    #Сбрка финального словаря предсказаний
    dict_of_neighbors = dict_join(dict_of_neigh, dict_of_tfidf_neighbors)
    return  dict_of_neighbors, targets

In [85]:
%%time
dict_final, targts = main_function()

CPU times: total: 28.4 s
Wall time: 27.8 s


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\den_m\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [86]:
len(dict_final)

1965

In [19]:
parser_final = parser_prep(parser)
products_final = products_prep(products)

In [457]:
parser_final

,key,price,dealer_id,name_new,article,dimension,quantity
0,546227,233.0,2,средство универсальный universal spray,,л,0.5
1,546408,175.0,2,концентрат multipower для мытьё пол цитрус,,л,1.0
2,546234,285.0,2,средство для чистка люстр universal anti dust,,л,0.5
3,651258,362.0,2,удалитель ржавчина rust remover,023-05,л,0.5
4,546355,205.0,2,средство мыть для баня и сауна multipower wood,,л,1.0
...,...,...,...,...,...,...,...
2996,717999545,1350.0,17,строительный антисептик глубокий проникновение,,кг,9.9
2997,718000129,1282.0,17,огнебиозащита для древесина огнебио prof 1 выс...,,кг,12.0
2998,900996549,294.0,18,отбеливатель для древесина eco 50 готовый состав,,кг,1.0
2999,530170161,751.0,18,строительный антисептик,,кг,5.6


**Все что ниже - не успели обернуть в функции и закинуть в главную функцию**

In [130]:
dims = base_tfidf.shape[1]
n_cells = 1 # количество центроидов
quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

In [131]:
# подготовка к поиску
idx_l2.train(np.ascontiguousarray(base_tfidf.values).astype('float32'))
idx_l2.add(np.ascontiguousarray(base_tfidf.values).astype('float32'))

In [241]:
# создание словаря для нахождения индекса товара в базовом наборе данных
base_index = {k: v for k, v in enumerate(base_tfidf.index.to_list())}
vecs, idx = idx_l2.search(np.ascontiguousarray(query_tfidf.values).astype('float32'), 10)

In [305]:
vecs_train, vecs_test, targets_train, targets_test = train_test_split(vecs, targets, test_size=0.3, random_state=12345)
idx_train, idx_test= train_test_split(idx, test_size=0.3, random_state=12345)

print(vecs_train.shape)
print(vecs_test.shape)
print(idx_train.shape)
print(idx_test.shape)
print(targets_train.shape)
print(targets_test.shape)

(1121, 10)
(481, 10)
(1121, 10)
(481, 10)
(1121,)
(481,)


In [400]:
accuracy_aggregator(targets, idx, base_index,5)

79.27590511860174

In [170]:
targets_train

1213    328
674     341
826     130
1046    245
1437    207
       ... 
1444    234
129     389
1309    186
1509    320
482     153
Name: product_id, Length: 1121, dtype: int64

In [212]:
base_query_train = pd.DataFrame()
# для каждого запроса из обучающего набора данных
for i,x in zip(range(vecs_train.shape[0]), targets_train.index):
    # берем индексы FAISS для первых 15 рекомендаций
    df = pd.DataFrame(idx_train[i])
    # берем расстояния от запроса до рекомендаций FAISS
    df = pd.concat([df, pd.DataFrame(vecs_train[i])], axis=1)
    df.columns = ['idx', 'distance']
    
    # индекс вектора-запроса для получения его координат
    df['query_idx'] = x
    
    # индекс вектора рекомендаций экспертов
    df['true_answer_for_query_idx'] = targets_train[x]
    base_query_train = pd.concat([base_query_train, df], ignore_index=True)
    

In [213]:
base_query_train

,idx,distance,query_idx,true_answer_for_query_idx
0,203,0.000000,1213,328
1,71,0.432659,1213,328
2,185,0.533340,1213,328
3,188,0.647902,1213,328
4,439,0.693470,1213,328
...,...,...,...,...
16810,349,1.662705,482,153
16811,350,1.662705,482,153
16812,362,1.667416,482,153
16813,434,1.692205,482,153


In [214]:
base_query_test = pd.DataFrame()
for i,x in zip(range(vecs_test.shape[0]), targets_test.index):
    df = pd.DataFrame(idx_test[i])

    df = pd.concat([df, pd.DataFrame(vecs_test[i])], axis=1)
    df.columns = ['idx', 'distance']
    
    df['query_idx'] = x
    
    df['true_answer_for_query_idx'] = targets_test[x]
    base_query_test = pd.concat([base_query_test, df], ignore_index=True)

In [215]:
base_query_test

,idx,distance,query_idx,true_answer_for_query_idx
0,419,0.396210,1465,26
1,426,0.396210,1465,26
2,23,1.000000,1465,26
3,35,1.000000,1465,26
4,98,1.000000,1465,26
...,...,...,...,...
7210,200,1.587673,1600,321
7211,438,1.618829,1600,321
7212,440,1.618829,1600,321
7213,203,1.639271,1600,321


In [181]:
# функция для получения индекса рекомендаций в базовом наборе товаров по индексу FAISS
def get_base_idx(row):
    return base_index[row['idx']]

In [216]:
# по внутреннему индексу FAISS восстанавливаем индекс рекомендованного товара в базовом наборе
base_query_train['true_answer_for_idx'] = base_query_train.apply(get_base_idx, axis=1)
base_query_test['true_answer_for_idx'] = base_query_test.apply(get_base_idx, axis=1)

base_query_train['target'] = (base_query_train['true_answer_for_query_idx'] == base_query_train['true_answer_for_idx']).astype('int')
base_query_test['target'] = (base_query_test['true_answer_for_query_idx'] == base_query_test['true_answer_for_idx']).astype('int')

In [217]:
base_query_train

,idx,distance,query_idx,true_answer_for_query_idx,true_answer_for_idx,target
0,203,0.000000,1213,328,328,1
1,71,0.432659,1213,328,492,0
2,185,0.533340,1213,328,219,0
3,188,0.647902,1213,328,327,0
4,439,0.693470,1213,328,330,0
...,...,...,...,...,...,...
16810,349,1.662705,482,153,158,0
16811,350,1.662705,482,153,159,0
16812,362,1.667416,482,153,150,0
16813,434,1.692205,482,153,161,0


In [292]:
# OHE
encoder = OneHotEncoder()
try:
    new_demensions = encoder.fit_transform(base[['dimension']]).toarray()
    products_final_ohe = pd.concat([products_final, pd.DataFrame(data=new_demensions, columns=encoder.categories_[0])], axis=1).drop(['dimension'], axis=1)
except:
    pass
try:
    new_demensions = encoder.transform(query[['dimension']]).toarray()
    parser_final_ohe = pd.concat([parser_final, pd.DataFrame(data=new_demensions, columns=encoder.categories_[0])], axis=1).drop(['dimension'], axis=1)
except:
    pass

In [297]:
products_final_ohe

,id,article,cost,recommended_price,full_name,quantity,кг,л,шт,NaN
0,245,008-1,360.00,858.0,антисептик невымываемый для ответственный конс...,1.00,0.0,1.0,0.0,0.0
1,3,242-12,460.16,1075.0,антигололед 32 готовый состав,12.00,1.0,0.0,0.0,0.0
2,443,0024-06 с,307.00,644.0,герметик акриловый цвет сосна пакет для шов де...,0.60,0.0,1.0,0.0,0.0
3,147,305-2,157.73,342.0,кондиционер для бельё королевский ирис crystal...,2.00,0.0,1.0,0.0,0.0
4,502,0024-7 б,NaN,891.0,герметик акриловой цвет белый,7.00,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
491,127,152-5,849.78,1985.0,средство для уборка помещение после пожар с де...,5.00,0.0,1.0,0.0,0.0
492,160,289-1,201.00,436.0,гель для стирка шерсть шёлк и деликатный ткань...,1.00,0.0,1.0,0.0,0.0
493,74,192-05,108.60,254.0,cредство для чистка гриль и духовка cooky gril...,0.50,0.0,1.0,0.0,0.0
494,34,186-5,579.74,1354.0,профессиональный средство для мытьё пол с поли...,5.00,0.0,1.0,0.0,0.0


In [293]:
def features_to_siries(list_of_features):
    f=[]
    for c in list_of_features:
        f.append(c)
    return f

In [298]:
query_f['feats'] = query.apply(lambda x: features_to_siries([x['price'],x['quantity'],x['кг'],x['л'],x['шт'],x['nan']]), axis=1)
base_f['feats'] = base.apply(lambda x: features_to_siries([x['price'],x['quantity'],x['кг'],x['л'],x['шт'],x['nan']]), axis=1)

KeyError: 'кг'

In [ ]:
base_f

In [ ]:
'''
Альтернативный вариант не добил, вариант где у нас будут 2 колонки с собранными фичами по query и base
Выводило все ок, но потом наверху чтото поменял, слетело чтото, а потом уже не до этого было
'''
total_table_alternative = base_query_train.merge(train_f['feats'], 
                              how='left',
                              left_on='query_idx',
                              right_index=True).\
                        merge(base_f['feats'], 
                              how='left',
                              left_on='base_idx',
                              right_index=True)

In [221]:
total_table_train = base_query_train.merge(train.loc[:,['price','article','quantity','dimension']], 
                                          how='left',
                                          left_on='query_idx',
                                          right_index=True).\
                                    merge(base.loc[:,['recommended_price','article','quantity','dimension']], 
                                          how='left',
                                          left_on='true_answer_for_idx',
                                          right_index=True)
total_table_train

,idx,distance,query_idx,true_answer_for_query_idx,true_answer_for_idx,target,price,article_x,quantity_x,dimension_x,recommended_price,article_y,quantity_y,dimension_y
0,203,0.000000,1213,328,328,1,793.0,,0.9,л,901.0,058-09,0.9,л
1,71,0.432659,1213,328,492,0,793.0,,0.9,л,1728.0,м028-2,2.0,шт
2,185,0.533340,1213,328,219,0,793.0,,0.9,л,1706.0,058-2,2.0,л
3,188,0.647902,1213,328,327,0,793.0,,0.9,л,885.0,056-09,0.9,л
4,439,0.693470,1213,328,330,0,793.0,,0.9,л,813.0,059-09,0.9,л
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16810,349,1.662705,482,153,158,0,1064.0,290-3,3.0,л,436.0,287-1,1.0,л
16811,350,1.662705,482,153,159,0,1064.0,290-3,3.0,л,1070.0,287-3,3.0,л
16812,362,1.667416,482,153,150,0,1064.0,290-3,3.0,л,1041.0,248-3,3.0,л
16813,434,1.692205,482,153,161,0,1064.0,290-3,3.0,л,1070.0,289-3,3.0,л


In [222]:
total_table_test = base_query_test.merge(train.loc[:,['price','article','quantity','dimension']], 
                                          how='left',
                                          left_on='query_idx',
                                          right_index=True).\
                                    merge(base.loc[:,['recommended_price','article','quantity','dimension']], 
                                          how='left',
                                          left_on='true_answer_for_idx',
                                          right_index=True)

In [356]:
total_table_test

,idx,distance,query_idx,true_answer_for_query_idx,true_answer_for_idx,target,price,article_x,quantity_x,dimension_x,recommended_price,article_y,quantity_y,dimension_y
0,419,0.396210,1465,26,26,1,NaN,NaN,NaN,NaN,236.0,230-1,1.0,л
1,426,0.396210,1465,26,27,0,NaN,NaN,NaN,NaN,800.0,230-5,5.0,л
2,23,1.000000,1465,26,503,0,NaN,NaN,NaN,NaN,891.0,0024-7 о,0.0,NaN
3,35,1.000000,1465,26,504,0,NaN,NaN,NaN,NaN,891.0,w022-05,0.0,NaN
4,98,1.000000,1465,26,436,0,NaN,NaN,NaN,NaN,600.0,р1 09005,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7210,200,1.587673,1600,321,482,0,NaN,NaN,NaN,NaN,491.0,м034-2,0.0,NaN
7211,438,1.618829,1600,321,325,0,NaN,NaN,NaN,NaN,551.0,055-09,0.9,л
7212,440,1.618829,1600,321,326,0,NaN,NaN,NaN,NaN,1086.0,055-2,2.0,л
7213,203,1.639271,1600,321,328,0,NaN,NaN,NaN,NaN,901.0,058-09,0.9,л


In [225]:
total_table_train['quantity'] = (total_table_train['quantity_x'] == total_table_train['quantity_y']) * 1
total_table_train['dimension'] = (total_table_train['dimension_x'] == total_table_train['dimension_y']) * 1
total_table_train['diff_price'] = np.abs(total_table_train['price'] - total_table_train['recommended_price'])
#total_table['article'] = total_table['article_x'].apply(lambda x: 'unknown' if type(x['article_x']) == '' else ((x['article_x'] == x['article_y']) * 1))
total_table_train['article'] = total_table_train['article_x'].isna()
for i in range(total_table_train.shape[0]):
    if total_table_train['article_x'][i] == '':
        total_table_train['article'][i] = 'unknown'
    else:
        total_table_train['article'][i] = str((total_table_train['article_x'][i] == total_table_train['article_y'][i]) * 1)

C:\Users\den_m\AppData\Local\Temp\ipykernel_14664\2525029445.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_table_train['article'][i]='unknown'


In [226]:
total_table_train

,idx,distance,query_idx,true_answer_for_query_idx,true_answer_for_idx,target,price,article_x,quantity_x,dimension_x,recommended_price,article_y,quantity_y,dimension_y,quantity,dimension,diff_price,article
0,203,0.000000,1213,328,328,1,793.0,,0.9,л,901.0,058-09,0.9,л,1,1,108.0,unknown
1,71,0.432659,1213,328,492,0,793.0,,0.9,л,1728.0,м028-2,2.0,шт,0,0,935.0,unknown
2,185,0.533340,1213,328,219,0,793.0,,0.9,л,1706.0,058-2,2.0,л,0,1,913.0,unknown
3,188,0.647902,1213,328,327,0,793.0,,0.9,л,885.0,056-09,0.9,л,1,1,92.0,unknown
4,439,0.693470,1213,328,330,0,793.0,,0.9,л,813.0,059-09,0.9,л,1,1,20.0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16810,349,1.662705,482,153,158,0,1064.0,290-3,3.0,л,436.0,287-1,1.0,л,0,1,628.0,0
16811,350,1.662705,482,153,159,0,1064.0,290-3,3.0,л,1070.0,287-3,3.0,л,1,1,6.0,0
16812,362,1.667416,482,153,150,0,1064.0,290-3,3.0,л,1041.0,248-3,3.0,л,1,1,23.0,0
16813,434,1.692205,482,153,161,0,1064.0,290-3,3.0,л,1070.0,289-3,3.0,л,1,1,6.0,0


In [227]:
total_table_train = total_table_train.drop(['article_x', 'article_y', 'quantity_x',
                      'quantity_y', 'dimension_x','dimension_y',
                      'recommended_price', 'price'], axis=1)

In [228]:
total_table_train

,idx,distance,query_idx,true_answer_for_query_idx,true_answer_for_idx,target,quantity,dimension,diff_price,article
0,203,0.000000,1213,328,328,1,1,1,108.0,unknown
1,71,0.432659,1213,328,492,0,0,0,935.0,unknown
2,185,0.533340,1213,328,219,0,0,1,913.0,unknown
3,188,0.647902,1213,328,327,0,1,1,92.0,unknown
4,439,0.693470,1213,328,330,0,1,1,20.0,unknown
...,...,...,...,...,...,...,...,...,...,...
16810,349,1.662705,482,153,158,0,0,1,628.0,0
16811,350,1.662705,482,153,159,0,1,1,6.0,0
16812,362,1.667416,482,153,150,0,1,1,23.0,0
16813,434,1.692205,482,153,161,0,1,1,6.0,0


In [ ]:
total_table_train = total_table_train.drop(['idx', 'query_idx',
         'target_idx', 'base_idx'], axis=1)

In [ ]:
total_scaler = MinMaxScaler()
total_table[['distance', 'diff_price']] = total_scaler.fit_transform(total_table[['distance', 'diff_price']])

In [ ]:
scaler = MinMaxScaler()
train.columns = train.columns.astype('str')
base.columns = base.columns.astype('str')
base['price'] = base['recommended_price']
base_f = pd.DataFrame(scaler.fit_transform(base.loc[:,['price','quantity','кг','л','шт','nan']]), columns=scaler.feature_names_in_).set_index(base_tfidf.index)
train_f = pd.DataFrame(scaler.transform(train.loc[:,['price','quantity','кг','л','шт','nan']]), columns=scaler.feature_names_in_)
#base_f = base_f.dropna(subset=['price'])
base_f

In [236]:
total_encoder = OneHotEncoder()

dem = total_encoder.fit_transform(total_table_train[['article']]).toarray()
total_table_train = pd.concat([total_table_train, pd.DataFrame(data=dem, columns=total_encoder.categories_[0])], axis=1).drop(['article'], axis=1)

In [237]:
total_table_train = total_table_train.rename({"0":"art_0","1":"art_1", "unknown":"art_unknown"}, axis='columns')

In [238]:
total_table_train

,idx,distance,query_idx,true_answer_for_query_idx,true_answer_for_idx,target,quantity,dimension,diff_price,art_0,art_1,art_unknown
0,203,0.000000,1213,328,328,1,1,1,108.0,0.0,0.0,1.0
1,71,0.432659,1213,328,492,0,0,0,935.0,0.0,0.0,1.0
2,185,0.533340,1213,328,219,0,0,1,913.0,0.0,0.0,1.0
3,188,0.647902,1213,328,327,0,1,1,92.0,0.0,0.0,1.0
4,439,0.693470,1213,328,330,0,1,1,20.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16810,349,1.662705,482,153,158,0,0,1,628.0,1.0,0.0,0.0
16811,350,1.662705,482,153,159,0,1,1,6.0,1.0,0.0,0.0
16812,362,1.667416,482,153,150,0,1,1,23.0,1.0,0.0,0.0
16813,434,1.692205,482,153,161,0,1,1,6.0,1.0,0.0,0.0


In [231]:
#Таблица, в которой отразили соответствие/несоответствие показателей(артикул, объем/количество, ед. измерения, разница цены) в связке query-base
total_table

,idx,distance,query_idx,true_answer_for_query_idx,true_answer_for_idx,target,quantity,dimension,diff_price,0,1,unknown
0,203,0.000000,1213,328,328,1,1,1,108.0,0.0,0.0,1.0
1,71,0.432659,1213,328,492,0,0,0,935.0,0.0,0.0,1.0
2,185,0.533340,1213,328,219,0,0,1,913.0,0.0,0.0,1.0
3,188,0.647902,1213,328,327,0,1,1,92.0,0.0,0.0,1.0
4,439,0.693470,1213,328,330,0,1,1,20.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16810,349,1.662705,482,153,158,0,0,1,628.0,1.0,0.0,0.0
16811,350,1.662705,482,153,159,0,1,1,6.0,1.0,0.0,0.0
16812,362,1.667416,482,153,150,0,1,1,23.0,1.0,0.0,0.0
16813,434,1.692205,482,153,161,0,1,1,6.0,1.0,0.0,0.0


In [239]:
#Передаю файл на дальнейшую работу в тетрадке prosept_model
total_table.to_csv('tt_test_den.csv')